In [4]:
import pandas as pd
import numpy as np, os
import matplotlib.pyplot as plt

from pathlib import Path

from vivarium import InteractiveContext
import scipy

!date
!whoami
!pwd

Wed Nov  5 15:25:33 PST 2025
ndbs
/mnt/share/code/ndbs/vivarium_research_alzheimers/verification_and_validation


In [3]:
! pip list | grep vivarium

vivarium                  3.6.0
vivarium_build_utils      2.0.12
vivarium_cluster_tools    2.1.23
vivarium_csu_alzheimers   0.1.dev58+g263eb3d9b /mnt/share/code/ndbs/vivarium_csu_alzheimers
vivarium_dependencies     1.0.1
vivarium_public_health    4.3.12


In [5]:
! pip freeze | grep vivarium

vivarium==3.6.0
vivarium_build_utils==2.0.12
vivarium_cluster_tools==2.1.23
-e git+https://github.com/ihmeuw/vivarium_csu_alzheimers.git@263eb3d9b80b13735a7380bb892430eaba92603a#egg=vivarium_csu_alzheimers
vivarium_dependencies==1.0.1
vivarium_public_health==4.3.12


In [6]:
import vivarium_csu_alzheimers
# model_specifications/model_spec.yaml is the typical location within the engineering repo for the
# default spec to live. Ask the engineers if you can't find it.
path = Path(vivarium_csu_alzheimers.__file__.replace('__init__.py', 'model_specifications/model_spec.yaml'))
print(path)

!cat $path

/mnt/share/code/ndbs/vivarium_csu_alzheimers/src/vivarium_csu_alzheimers/model_specifications/model_spec.yaml
components:
    vivarium_csu_alzheimers:
      components:
          - ResultsStratifier()
          - NewSimulantsObserver()
          - AlzheimersIncidence()
          - AlzheimersPopulation('population.scaling_factor')
          - Alzheimers()
          - Testing()
          - BaselineTestingObserver()
          - BBBMTestingObserver()
          - Treatment()
          - TreatmentObserver()
          - TreatmentRiskEffect('cause.alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate')
            
    vivarium_public_health:
        population:
            - Mortality()
        results:
            - DisabilityObserver()
            - MortalityObserver()
            - DiseaseObserver('alzheimers_disease_and_other_dementias')

configuration:
    input_data:
        input_draw_number: 0
        artifact_path: '/mnt/team/simulation_

# Customize the model specification by deleting observers

In [16]:
from vivarium.framework.configuration import build_model_specification
custom_model_specification = build_model_specification(path)

# Delete mortality, morbidity and disease observers
print(custom_model_specification.components.vivarium_public_health.results)
del custom_model_specification.components.vivarium_public_health.results

# Filter out components with 'Observer' or 'Stratifier' in the name
observers = [
    c for c in custom_model_specification.components.vivarium_csu_alzheimers.components
    if 'Observer' in c or 'Stratifier' in c
]
print(observers)
custom_model_specification.components.vivarium_csu_alzheimers.components = [
    c for c in custom_model_specification.components.vivarium_csu_alzheimers.components
    if c not in observers
]

# Create a different model specification for each scenario
baseline_spec = build_model_specification(
    custom_model_specification,
    configuration={'intervention': {'scenario': 'baseline'}}
)
testing_spec = build_model_specification(
    custom_model_specification,
    configuration={'intervention': {'scenario': 'bbbm_testing'}}
)
treatment_spec = build_model_specification(
    custom_model_specification,
    configuration={'intervention': {'scenario': 'bbbm_testing_and_treatment'}}
)

# Set scenario to intervention instead of baseline
# custom_model_specification.configuration.intervention.scenario = 'bbbm_testing_and_treatment'

custom_model_specification.components.vivarium_csu_alzheimers.components

['DisabilityObserver()', 'MortalityObserver()', "DiseaseObserver('alzheimers_disease_and_other_dementias')"]
['ResultsStratifier()', 'NewSimulantsObserver()', 'BaselineTestingObserver()', 'BBBMTestingObserver()', 'TreatmentObserver()']


['AlzheimersIncidence()',
 "AlzheimersPopulation('population.scaling_factor')",
 'Alzheimers()',
 'Testing()',
 'Treatment()',
 "TreatmentRiskEffect('cause.alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate')"]

# Create simulations using the customized model specs

In [25]:
baseline = InteractiveContext(baseline_spec)
testing = InteractiveContext(testing_spec)

2025-11-05 15:47:51.383 | INFO     | simulation_3-artifact_manager:79 - Running simulation from artifact located at /mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf.
2025-11-05 15:47:51.385 | INFO     | simulation_3-artifact_manager:80 - Artifact base filter terms are ['draw == 0'].
2025-11-05 15:47:51.385 | INFO     | simulation_3-artifact_manager:81 - Artifact additional filter terms are None.
2025-11-05 15:47:53.901 | WARNING  | simulation_3-values_manager:55 - Unsourced pipelines: ['all_causes.disability_weight']
2025-11-05 15:47:53.905 | WARNING  | simulation_3-resource_manager:176 - Resource stream.alzheimers_disease_and_other_dementias_initial_states is not produced by any component but is needed to compute (column.alzheimers_disease_and_other_dementias).
2025-11-05 15:47:53.906 | WARNING  | simulation_3-resource_manager:176 - Resource stream.treatment_initial_states is not produced by any component but is needed to 

In [ ]:
# Check whether initial populations are the same
print(baseline.current_time, testing.current_time)
bpop = baseline.get_population()
tpop = testing.get_population()
bpop.equals(tpop)

2022-01-01 00:00:00 2022-01-01 00:00:00


True

In [ ]:
# Take a step and check again
baseline.step()
testing.step()

print(baseline.current_time, testing.current_time)
bpop = baseline.get_population()
tpop = testing.get_population()
bpop.equals(tpop)

2025-11-05 15:49:39.170 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2022-01-01 00:00:00
2025-11-05 15:49:43.495 | INFO     | simulation_4 - vivarium.framework.engine:284 - 2022-01-01 00:00:00
2022-07-02 00:00:00 2022-07-02 00:00:00


False

In [29]:
bpop.compare(tpop)

bbbm_test_ever_eligible      
                         self other
0                       False  True
4                       False  True
7                       False  True
24                      False  True
25                      False  True
...                       ...   ...
19978                   False  True
19979                   False  True
19991                   False  True
19996                   False  True
19997                   False  True

[3766 rows x 2 columns]

In [30]:
# Take another step and check again
baseline.step()
testing.step()

print(baseline.current_time, testing.current_time)
bpop = baseline.get_population()
tpop = testing.get_population()
bpop.compare(tpop)

2025-11-05 15:51:40.121 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2022-07-02 00:00:00
2025-11-05 15:51:44.574 | INFO     | simulation_4 - vivarium.framework.engine:284 - 2022-07-02 00:00:00
2022-12-31 00:00:00 2022-12-31 00:00:00


bbbm_test_ever_eligible      
                         self other
0                       False  True
4                       False  True
7                       False  True
24                      False  True
25                      False  True
...                       ...   ...
21361                   False  True
21362                   False  True
21363                   False  True
21364                   False  True
21365                   False  True

[4699 rows x 2 columns]

# Advance both sims until right before testing starts in 2030

In [31]:

baseline.run_until(pd.Timestamp('2029-01-01'))
testing.run_until(pd.Timestamp('2029-01-01'))

2025-11-05 15:52:51.592 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2022-12-31 00:00:00
2025-11-05 15:52:55.819 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2023-07-01 00:00:00
2025-11-05 15:53:00.176 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2023-12-30 00:00:00
2025-11-05 15:53:04.534 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2024-06-29 00:00:00
2025-11-05 15:53:09.117 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2024-12-28 00:00:00
2025-11-05 15:53:13.690 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2025-06-28 00:00:00
2025-11-05 15:53:18.262 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2025-12-27 00:00:00
2025-11-05 15:53:22.932 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2026-06-27 00:00:00
2025-11-05 15:53:27.929 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2026-12-26 00:00:00
2025-11-05 15:53:33.408 | INFO     | simulation_3 - vivarium.fra

2025-11-05 15:53:55.727 | INFO     | simulation_4 - vivarium.framework.engine:284 - 2022-12-31 00:00:00
2025-11-05 15:53:59.969 | INFO     | simulation_4 - vivarium.framework.engine:284 - 2023-07-01 00:00:00
2025-11-05 15:54:04.357 | INFO     | simulation_4 - vivarium.framework.engine:284 - 2023-12-30 00:00:00
2025-11-05 15:54:08.792 | INFO     | simulation_4 - vivarium.framework.engine:284 - 2024-06-29 00:00:00
2025-11-05 15:54:13.235 | INFO     | simulation_4 - vivarium.framework.engine:284 - 2024-12-28 00:00:00
2025-11-05 15:54:17.816 | INFO     | simulation_4 - vivarium.framework.engine:284 - 2025-06-28 00:00:00
2025-11-05 15:54:22.610 | INFO     | simulation_4 - vivarium.framework.engine:284 - 2025-12-27 00:00:00
2025-11-05 15:54:27.515 | INFO     | simulation_4 - vivarium.framework.engine:284 - 2026-06-27 00:00:00
2025-11-05 15:54:32.902 | INFO     | simulation_4 - vivarium.framework.engine:284 - 2026-12-26 00:00:00
2025-11-05 15:54:38.278 | INFO     | simulation_4 - vivarium.fra

In [33]:
# Ok good, the difference is still only in the 'bbbm_test_ever_eligible'
# column
print(baseline.current_time, testing.current_time)
bpop = baseline.get_population()
tpop = testing.get_population()
bpop.compare(tpop)

2029-06-23 00:00:00 2029-06-23 00:00:00


bbbm_test_ever_eligible      
                         self other
0                       False  True
4                       False  True
7                       False  True
24                      False  True
25                      False  True
...                       ...   ...
43049                   False  True
43050                   False  True
43051                   False  True
43052                   False  True
43053                   False  True

[18341 rows x 2 columns]

# Take another step in 2029

In [34]:
baseline.step()
testing.step()

print(baseline.current_time, testing.current_time)
bpop = baseline.get_population()
tpop = testing.get_population()

bpop.compare(tpop)

2025-11-05 16:03:01.408 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2029-06-23 00:00:00
2025-11-05 16:03:09.004 | INFO     | simulation_4 - vivarium.framework.engine:284 - 2029-06-23 00:00:00
2029-12-22 00:00:00 2029-12-22 00:00:00


bbbm_test_ever_eligible      
                         self other
0                       False  True
4                       False  True
7                       False  True
24                      False  True
25                      False  True
...                       ...   ...
44858                   False  True
44859                   False  True
44860                   False  True
44862                   False  True
44863                   False  True

[19449 rows x 2 columns]

# Take the first step where testing can occur

In [36]:
baseline.step()
testing.step()

print(baseline.current_time, testing.current_time)
bpop = baseline.get_population()
tpop = testing.get_population()

bpop.compare(tpop)

2025-11-05 16:04:19.728 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2029-12-22 00:00:00
2025-11-05 16:04:26.327 | INFO     | simulation_4 - vivarium.framework.engine:284 - 2029-12-22 00:00:00
2030-06-22 00:00:00 2030-06-22 00:00:00


bbbm_test_ever_eligible       testing_state       bbbm_test_result  \
                         self other          self other             self   
0                       False  True           NaN   NaN              NaN   
4                       False  True           NaN   NaN              NaN   
7                       False  True           NaN   NaN              NaN   
24                      False  True           NaN   NaN              NaN   
25                      False  True           NaN   NaN              NaN   
...                       ...   ...           ...   ...              ...   
48513                   False  True    not_tested  bbbm       not_tested   
48522                   False  True    not_tested  bbbm       not_tested   
48558                   False  True    not_tested  bbbm       not_tested   
48575                   False  True    not_tested  bbbm       not_tested   
48578                   False  True    not_tested  bbbm       not_tested   

                bbbm_test_date            no_effect_never_treated_event_time  \
          other           self      other                               self   
0           NaN            NaT        NaT                                NaT   
4           NaN            NaT        NaT                                NaT   
7           NaN            NaT        NaT                                NaT   
24          NaN            NaT        NaT                                NaT   
25          NaN            NaT        NaT                                NaT   
...         ...            ...        ...                                ...   
48513  positive            NaT 2030-06-22                                NaT   
48522  negative            NaT 2030-06-22                                NaT   
48558  positive            NaT 2030-06-22                                NaT   
48575  positive            NaT 2030-06-22                                NaT   
48578  positive            NaT 2030-06-22                                NaT   

                                 treatment                           \
           other                      self                    other   
0            NaT                       NaN                      NaN   
4            NaT                       NaN                      NaN   
7            NaT                       NaN                      NaN   
24           NaT                       NaN                      NaN   
25           NaT                       NaN                      NaN   
...          ...                       ...                      ...   
48513 2030-06-22  susceptible_to_treatment  no_effect_never_treated   
48522        NaT                       NaN                      NaN   
48558 2030-06-22  susceptible_to_treatment  no_effect_never_treated   
48575 2030-06-22  susceptible_to_treatment  no_effect_never_treated   
48578 2030-06-22  susceptible_to_treatment  no_effect_never_treated   

      no_effect_never_treated_event_count       positive_test_event_count  \
                                     self other                      self   
0                                     NaN   NaN                       NaN   
4                                     NaN   NaN                       NaN   
7                                     NaN   NaN                       NaN   
24                                    NaN   NaN                       NaN   
25                                    NaN   NaN                       NaN   
...                                   ...   ...                       ...   
48513                                 0.0   1.0                       NaN   
48522                                 NaN   NaN                       NaN   
48558                                 0.0   1.0                       NaN   
48575                                 0.0   1.0                       NaN   
48578                                 0.0   1.0                       NaN   

            positive_test_event_time        
     

In [38]:
bpop.columns

Index(['tracked', 'entrance_time', 'sex', 'alive', 'age', 'exit_time',
       'location', 'cause_of_death', 'years_of_life_lost',
       'alzheimers_disease_and_other_dementias', 'bbbm_entrance_time',
       'alzheimers_blood_based_biomarker_state_event_count',
       'alzheimers_blood_based_biomarker_state_event_time',
       'alzheimers_mild_cognitive_impairment_state_event_count',
       'alzheimers_mild_cognitive_impairment_state_event_time',
       'alzheimers_disease_state_event_count',
       'alzheimers_disease_state_event_time', 'bbbm_test_ever_eligible',
       'testing_propensity', 'testing_state', 'bbbm_test_result',
       'bbbm_test_date', 'treatment_propensity',
       'no_effect_never_treated_event_time',
       'waiting_for_treatment_event_time', 'waiting_for_treatment_event_count',
       'treatment', 'no_effect_never_treated_event_count',
       'susceptible_to_treatment_event_count',
       'susceptible_to_treatment_event_time', 'positive_test_event_count',
       '

In [48]:
[c for c in bpop.columns if 'test' in c]

['bbbm_test_ever_eligible',
 'testing_propensity',
 'testing_state',
 'bbbm_test_result',
 'bbbm_test_date',
 'positive_test_event_count',
 'positive_test_event_time']

In [56]:
[v for v in baseline.list_values() if 'test' in v]

[]

In [41]:
tpop.loc[tpop.testing_state != 'not_tested', 'testing_state']

7         pet
9         pet
16        csf
20        pet
22        csf
         ... 
48513    bbbm
48522    bbbm
48558    bbbm
48575    bbbm
48578    bbbm
Name: testing_state, Length: 8581, dtype: object

In [44]:
bpop.loc[tpop.testing_state != 'not_tested', 'testing_state']

7               pet
9               pet
16              csf
20              pet
22              csf
            ...    
48513    not_tested
48522    not_tested
48558    not_tested
48575    not_tested
48578    not_tested
Name: testing_state, Length: 8581, dtype: object

In [45]:
tpop.loc[bpop.testing_state != 'not_tested', 'testing_state']

7        pet
9        pet
16       csf
20       pet
22       csf
        ... 
46585    pet
46592    csf
46668    pet
46713    pet
46854    pet
Name: testing_state, Length: 7550, dtype: object

In [46]:
bbbm = tpop.testing_state == 'bbbm'
bpop.loc[bbbm]

,tracked,entrance_time,sex,alive,age,exit_time,location,cause_of_death,years_of_life_lost,alzheimers_disease_and_other_dementias,...,full_effect_short_event_count,full_effect_short_event_time,waning_effect_long_event_count,waning_effect_long_event_time,waning_effect_short_event_time,waning_effect_short_event_count,no_effect_after_long_event_time,no_effect_after_long_event_count,no_effect_after_short_event_count,no_effect_after_short_event_time
746,True,2021-07-03,Female,alive,69.473392,NaT,United States of America,not_dead,0.000000,alzheimers_blood_based_biomarker_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
990,True,2021-07-03,Male,dead,72.285205,2024-06-29,United States of America,other_causes,20.201015,alzheimers_blood_based_biomarker_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
1074,True,2021-07-03,Female,alive,72.332691,NaT,United States of America,not_dead,0.000000,alzheimers_blood_based_biomarker_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
1086,True,2021-07-03,Female,alive,70.034417,NaT,United States of America,not_dead,0.000000,alzheimers_blood_based_biomarker_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
1223,True,2021-07-03,Male,dead,71.924894,2022-12-31,United States of America,other_causes,20.504479,alzheimers_blood_based_biomarker_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48513,True,2029-12-22,Male,alive,78.433719,NaT,NaN,not_dead,0.000000,alzheimers_blood_based_biomarker_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
48522,True,2029-12-22,Male,alive,77.914145,NaT,NaN,not_dead,0.000000,alzheimers_blood_based_biomarker_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
48558,True,2029-12-22,Male,alive,77.371178,NaT,NaN,not_dead,0.000000,alzheimers_blood_based_biomarker_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
48575,True,2029-12-22,Male,alive,79.104005,NaT,NaN,not_dead,0.000000,alzheimers_blood_based_biomarker_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT


In [47]:
tpop.loc[bbbm]

,tracked,entrance_time,sex,alive,age,exit_time,location,cause_of_death,years_of_life_lost,alzheimers_disease_and_other_dementias,...,full_effect_short_event_count,full_effect_short_event_time,waning_effect_long_event_count,waning_effect_long_event_time,waning_effect_short_event_time,waning_effect_short_event_count,no_effect_after_long_event_time,no_effect_after_long_event_count,no_effect_after_short_event_count,no_effect_after_short_event_time
746,True,2021-07-03,Female,alive,69.473392,NaT,United States of America,not_dead,0.000000,alzheimers_blood_based_biomarker_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
990,True,2021-07-03,Male,dead,72.285205,2024-06-29,United States of America,other_causes,20.201015,alzheimers_blood_based_biomarker_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
1074,True,2021-07-03,Female,alive,72.332691,NaT,United States of America,not_dead,0.000000,alzheimers_blood_based_biomarker_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
1086,True,2021-07-03,Female,alive,70.034417,NaT,United States of America,not_dead,0.000000,alzheimers_blood_based_biomarker_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
1223,True,2021-07-03,Male,dead,71.924894,2022-12-31,United States of America,other_causes,20.504479,alzheimers_blood_based_biomarker_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48513,True,2029-12-22,Male,alive,78.433719,NaT,NaN,not_dead,0.000000,alzheimers_blood_based_biomarker_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
48522,True,2029-12-22,Male,alive,77.914145,NaT,NaN,not_dead,0.000000,alzheimers_blood_based_biomarker_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
48558,True,2029-12-22,Male,alive,77.371178,NaT,NaN,not_dead,0.000000,alzheimers_blood_based_biomarker_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
48575,True,2029-12-22,Male,alive,79.104005,NaT,NaN,not_dead,0.000000,alzheimers_blood_based_biomarker_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT


# Define a function to get columns and pipeline values we want from the population

In [61]:
# List columns we want to keep
columns = [
    'age', 'alive', 'bbbm_test_result', 'treatment',
    'alzheimers_disease_and_other_dementias',
    'bbbm_entrance_time',
] + [c for c in baseline.get_population().columns if 'test' in c]
# Rename some columns with shorter names if desired
columns_to_short_names = {c: c for c in columns} | {
    'alzheimers_disease_and_other_dementias': 'ad_state',
}
# Simultaneously list pipeline values we want and map them to shorter
# names
values_to_short_names = {
    'alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.transition_rate':
        'mci_inc_rate',
    'treatment_on_alzheimers_blood_based_biomarker_state_to_alzheimers_mild_cognitive_impairment_state.relative_risk':
        'relative_risk',
}

def get_pop_columns(
        sim,
        columns_to_short_names=columns_to_short_names,
        values_to_short_names=values_to_short_names,
    ):
    """Get the specified subset of population columns concatenated with
    the specified pipeline values, with columns for pipeline values
    renamed according to the names in the provided "short names"
    dictionary.
    """
    pop = sim.get_population()
    pipeline_values = [
        sim.get_value(v)(pop.index).rename(short_name)
        for v, short_name in values_to_short_names.items()]
    pop_columns = (
        pop[columns_to_short_names.keys()]
        .rename(columns=columns_to_short_names)
        .join(pipeline_values)
        # Add current time to the dataframe
        .assign(current_time=sim.current_time)
    )
    return pop_columns

def take_steps(
        sim,
        n,
        columns_to_short_names=columns_to_short_names,
        values_to_short_names=values_to_short_names,
    ):
    """Take n simulation steps, recording the specified population
    columns and values at each step, and return a list of the population
    tables at each step. The resulting list will have n+1 population
    tables.
    """
    population_trace = [
        get_pop_columns(sim, columns_to_short_names, values_to_short_names)]
    for _ in range(n):
        sim.step()
        population_trace.append(get_pop_columns(
            sim, columns_to_short_names, values_to_short_names))
    return population_trace

bpop1 = get_pop_columns(baseline).drop(columns='bbbm_test_ever_eligible')
bpop1

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,testing_propensity,testing_state,bbbm_test_date,positive_test_event_count,positive_test_event_time,mci_inc_rate,relative_risk,current_time
0,87.268568,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2019-09-10 05:16:48.497338228,0.516257,not_tested,NaT,0,NaT,0.099737,1.0,2030-06-22
1,91.219958,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2021-05-18 17:15:26.758385588,0.711627,not_tested,NaT,0,NaT,0.096070,1.0,2030-06-22
2,96.681074,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2016-11-21 22:42:06.776219113,0.827952,not_tested,NaT,0,NaT,0.104932,1.0,2030-06-22
3,84.152214,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2016-06-07 20:19:14.054748464,0.732312,not_tested,NaT,0,NaT,0.105699,1.0,2030-06-22
4,71.727825,dead,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2016-01-03 12:31:22.341663442,0.818851,not_tested,NaT,0,NaT,0.106400,1.0,2030-06-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48822,89.085473,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2029-12-22 00:00:00.000000000,0.130584,not_tested,NaT,0,NaT,0.050712,1.0,2030-06-22
48823,89.272447,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2029-12-22 00:00:00.000000000,0.740050,not_tested,NaT,0,NaT,0.050712,1.0,2030-06-22
48824,88.210390,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2029-12-22 00:00:00.000000000,0.276623,not_tested,NaT,0,NaT,0.050712,1.0,2030-06-22
48825,94.368270,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2029-12-22 00:00:00.000000000,0.778562,not_tested,NaT,0,NaT,0.050712,1.0,2030-06-22


In [62]:
tpop1 = get_pop_columns(testing).drop(columns='bbbm_test_ever_eligible')
tpop1

,age,alive,bbbm_test_result,treatment,ad_state,bbbm_entrance_time,testing_propensity,testing_state,bbbm_test_date,positive_test_event_count,positive_test_event_time,mci_inc_rate,relative_risk,current_time
0,87.268568,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2019-09-10 05:16:48.497338228,0.516257,not_tested,NaT,0,NaT,0.099737,1.0,2030-06-22
1,91.219958,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2021-05-18 17:15:26.758385588,0.711627,not_tested,NaT,0,NaT,0.096070,1.0,2030-06-22
2,96.681074,dead,not_tested,susceptible_to_treatment,alzheimers_mild_cognitive_impairment_state,2016-11-21 22:42:06.776219113,0.827952,not_tested,NaT,0,NaT,0.104932,1.0,2030-06-22
3,84.152214,dead,not_tested,susceptible_to_treatment,alzheimers_disease_state,2016-06-07 20:19:14.054748464,0.732312,not_tested,NaT,0,NaT,0.105699,1.0,2030-06-22
4,71.727825,dead,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2016-01-03 12:31:22.341663442,0.818851,not_tested,NaT,0,NaT,0.106400,1.0,2030-06-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48822,89.085473,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2029-12-22 00:00:00.000000000,0.130584,not_tested,NaT,0,NaT,0.050712,1.0,2030-06-22
48823,89.272447,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2029-12-22 00:00:00.000000000,0.740050,not_tested,NaT,0,NaT,0.050712,1.0,2030-06-22
48824,88.210390,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2029-12-22 00:00:00.000000000,0.276623,not_tested,NaT,0,NaT,0.050712,1.0,2030-06-22
48825,94.368270,alive,not_tested,susceptible_to_treatment,alzheimers_blood_based_biomarker_state,2029-12-22 00:00:00.000000000,0.778562,not_tested,NaT,0,NaT,0.050712,1.0,2030-06-22


In [63]:
bpop1.compare(tpop1)

bbbm_test_result                           treatment  \
                  self     other                      self   
746         not_tested  positive  susceptible_to_treatment   
990         not_tested  positive  susceptible_to_treatment   
1074        not_tested  negative                       NaN   
1086        not_tested  positive  susceptible_to_treatment   
1223        not_tested  positive  susceptible_to_treatment   
...                ...       ...                       ...   
48513       not_tested  positive  susceptible_to_treatment   
48522       not_tested  negative                       NaN   
48558       not_tested  positive  susceptible_to_treatment   
48575       not_tested  positive  susceptible_to_treatment   
48578       not_tested  positive  susceptible_to_treatment   

                               testing_state       bbbm_test_date             \
                         other          self other           self      other   
746    no_effect_never_treated    not_tested  bbbm            NaT 2030-06-22   
990    no_effect_never_treated    not_tested  bbbm            NaT 2030-06-22   
1074                       NaN    not_tested  bbbm            NaT 2030-06-22   
1086   no_effect_never_treated    not_tested  bbbm            NaT 2030-06-22   
1223   no_effect_never_treated    not_tested  bbbm            NaT 2030-06-22   
...                        ...           ...   ...            ...        ...   
48513  no_effect_never_treated    not_tested  bbbm            NaT 2030-06-22   
48522                      NaN    not_tested  bbbm            NaT 2030-06-22   
48558  no_effect_never_treated    not_tested  bbbm            NaT 2030-06-22   
48575  no_effect_never_treated    not_tested  bbbm            NaT 2030-06-22   
48578  no_effect_never_treated    not_tested  bbbm            NaT 2030-06-22   

      positive_test_event_count       positive_test_event_time             
                           self other                     self      other  
746                         0.0   1.0                      NaT 2030-06-22  
990                         0.0   1.0                      NaT 2030-06-22  
1074                        NaN   NaN                      NaT        NaT  
1086                        0.0   1.0                      NaT 2030-06-22  
1223                        0.0   1.0                      NaT 2030-06-22  
...                         ...   ...                      ...        ...  
48513                       NaN   NaN                      NaT        NaT  
48522                       NaN   NaN                      NaT        NaT  
48558                       NaN   NaN                      NaT        NaT  
48575                       NaN   NaN                      NaT        NaT  
48578                       NaN   NaN                      NaT        NaT  

[1031 rows x 12 columns]

# Check whether any CSF or PET tests got averted on the first testing timestep

No. Everyone who got a CSF or PET test in the baseline scenario also got
the same test in the testing scenario

In [64]:
baseline_csf_pet = bpop.testing_state.isin(['csf', 'pet'])
bpop.loc[baseline_csf_pet]

,tracked,entrance_time,sex,alive,age,exit_time,location,cause_of_death,years_of_life_lost,alzheimers_disease_and_other_dementias,...,full_effect_short_event_count,full_effect_short_event_time,waning_effect_long_event_count,waning_effect_long_event_time,waning_effect_short_event_time,waning_effect_short_event_count,no_effect_after_long_event_time,no_effect_after_long_event_count,no_effect_after_short_event_count,no_effect_after_short_event_time
7,True,2021-07-03,Female,dead,73.986766,2028-06-24,United States of America,other_causes,18.785646,alzheimers_disease_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
9,True,2021-07-03,Male,dead,84.154388,2024-12-28,United States of America,alzheimers_disease_state,11.466826,alzheimers_disease_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
16,True,2021-07-03,Female,alive,86.391784,NaT,United States of America,not_dead,0.000000,alzheimers_disease_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
20,True,2021-07-03,Female,dead,81.885820,2024-06-29,United States of America,other_causes,12.848163,alzheimers_mild_cognitive_impairment_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
22,True,2021-07-03,Female,dead,71.887990,2023-07-01,United States of America,alzheimers_disease_state,20.538726,alzheimers_disease_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46585,True,2029-06-23,Male,alive,78.479639,NaT,NaN,not_dead,0.000000,alzheimers_mild_cognitive_impairment_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
46592,True,2029-06-23,Male,alive,78.038541,NaT,NaN,not_dead,0.000000,alzheimers_mild_cognitive_impairment_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
46668,True,2029-06-23,Male,alive,76.223715,NaT,NaN,not_dead,0.000000,alzheimers_mild_cognitive_impairment_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT
46713,True,2029-06-23,Male,alive,83.403298,NaT,NaN,not_dead,0.000000,alzheimers_mild_cognitive_impairment_state,...,0,NaT,0,NaT,NaT,0,NaT,0,0,NaT


In [68]:
print(baseline.current_time, testing.current_time)
tpop.loc[baseline_csf_pet, 'testing_state'].unique()

2030-06-22 00:00:00 2030-06-22 00:00:00


array(['pet', 'csf'], dtype=object)

In [67]:
bpop.loc[baseline_csf_pet, 'testing_state'].compare(tpop.loc[baseline_csf_pet, 'testing_state'])

,self,other


# Take another step and check again

Now there are 69 people who had CSF/PET in baseline on this time step,
but have BBBM on the next time step

In [ ]:
# save current populations
bpop0, tpop0 = bpop, tpop
# People who had CSF/PET in baseline at time 0
baseline_csf_pet0 = bpop0.testing_state.isin(['csf', 'pet'])

In [70]:
baseline.step()
testing.step()

print(baseline.current_time, testing.current_time)
bpop = baseline.get_population()
tpop = testing.get_population()

2025-11-05 16:26:58.082 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2030-06-22 00:00:00
2025-11-05 16:27:06.040 | INFO     | simulation_4 - vivarium.framework.engine:284 - 2030-06-22 00:00:00
2030-12-21 00:00:00 2030-12-21 00:00:00


In [ ]:
# People who had CSF/PET in baseline at time 1
baseline_csf_pet = bpop.testing_state.isin(['csf', 'pet'])

print(baseline.current_time, testing.current_time)
tpop.loc[baseline_csf_pet, 'testing_state'].unique()

bpop.loc[baseline_csf_pet, 'testing_state'].compare(tpop.loc[baseline_csf_pet, 'testing_state'])

2030-12-21 00:00:00 2030-12-21 00:00:00


,self,other
11841,pet,bbbm
12016,pet,bbbm
21661,pet,bbbm
21750,pet,bbbm
21922,csf,bbbm
...,...,...
47103,pet,bbbm
47555,pet,bbbm
47571,pet,bbbm
47684,pet,bbbm


In [81]:
print(baseline_csf_pet0.sum())
print(baseline_csf_pet.sum())
print((baseline_csf_pet0 & baseline_csf_pet).sum())
print((baseline_csf_pet0 ^ baseline_csf_pet).sum())

7550
7934
7550
384


In [ ]:
# Check that everyone who had a CSF or PET test on step 0 still has it
# on step 1 (in the baseline scenario)
baseline_csf_pet.loc[(baseline_csf_pet0 & baseline_csf_pet)].equals(baseline_csf_pet0.loc[baseline_csf_pet0])

True

In [98]:
# Simulants who got a CSF/PET test on step 1 and didn't have one on step
# 0 (in the baseline scenario)
got_csf_pet_at_time_1 = baseline_csf_pet0 ^ baseline_csf_pet
baseline_csf_pet.loc[got_csf_pet_at_time_1]

190      True
970      True
1073     True
2121     True
2259     True
         ... 
48359    True
48380    True
48482    True
48598    True
48730    True
Name: testing_state, Length: 384, dtype: bool

In [99]:
# At time 1, these simulants had a CSF/PET test
print(bpop.loc[got_csf_pet_at_time_1, 'testing_state'].unique())
bpop.loc[got_csf_pet_at_time_1, 'testing_state']

['csf' 'pet']


190      csf
970      csf
1073     csf
2121     pet
2259     csf
        ... 
48359    pet
48380    csf
48482    pet
48598    pet
48730    pet
Name: testing_state, Length: 384, dtype: object

In [101]:
# At time 0, these simulants had not been tested
print(bpop0.loc[got_csf_pet_at_time_1, 'testing_state'].unique())
bpop0.loc[got_csf_pet_at_time_1, 'testing_state']

['not_tested']


190      not_tested
970      not_tested
1073     not_tested
2121     not_tested
2259     not_tested
            ...    
48359    not_tested
48380    not_tested
48482    not_tested
48598    not_tested
48730    not_tested
Name: testing_state, Length: 384, dtype: object

In [102]:
tpop0.loc[got_csf_pet_at_time_1, 'testing_state']

190      not_tested
970      not_tested
1073     not_tested
2121     not_tested
2259     not_tested
            ...    
48359    not_tested
48380          bbbm
48482          bbbm
48598    not_tested
48730    not_tested
Name: testing_state, Length: 384, dtype: object

In [108]:
tpop.loc[got_csf_pet_at_time_1, 'testing_state']

190       csf
970       csf
1073      csf
2121      pet
2259      csf
         ... 
48359     pet
48380    bbbm
48482     pet
48598     pet
48730     pet
Name: testing_state, Length: 384, dtype: object

In [112]:
tpop0.loc[got_csf_pet_at_time_1, 'testing_state'].value_counts()

testing_state
not_tested    304
bbbm           80
Name: count, dtype: int64

In [111]:
tpop.loc[got_csf_pet_at_time_1, 'testing_state'].value_counts()

testing_state
pet     210
csf     105
bbbm     69
Name: count, dtype: int64

In [ ]:
# Simulant 48380 got a CSF test on step 1 in the baseline scenario. In
# the testing scenario, they got a positive BBBM test on step 0, thus
# avoiding getting a CSF test on the next step.
pd.concat([bpop0.loc[48380], bpop.loc[48380], tpop0.loc[48380], tpop.loc[48380]],
    axis=1,
    keys=[('baseline', 0), ('baseline', 1), ('testing', 0), ('testing', 1)],
    names=['scenario', 'step']
    )

scenario                                                                          baseline  \
step                                                                                     0   
tracked                                                                               True   
entrance_time                                                          2029-12-22 00:00:00   
sex                                                                                   Male   
alive                                                                                alive   
age                                                                               73.90981   
exit_time                                                                              NaT   
location                                                                               NaN   
cause_of_death                                                                    not_dead   
years_of_life_lost                                                                     0.0   
alzheimers_disease_and_other_dementias              alzheimers_blood_based_biomarker_state   
bbbm_entrance_time                                                     2029-12-22 00:00:00   
alzheimers_blood_based_biomarker_state_event_count                                       0   
alzheimers_blood_based_biomarker_state_event_time                      2029-12-22 00:00:00   
alzheimers_mild_cognitive_impairment_state_even...                                       0   
alzheimers_mild_cognitive_impairment_state_even...                                     NaT   
alzheimers_disease_state_event_count                                                     0   
alzheimers_disease_state_event_time                                                    NaT   
bbbm_test_ever_eligible                                                              False   
testing_propensity                                                                 0.06354   
testing_state                                                                   not_tested   
bbbm_test_result                                                                not_tested   
bbbm_test_date                                                                         NaT   
treatment_propensity                                                              0.951688   
no_effect_never_treated_event_time                                                     NaT   
waiting_for_treatment_event_time                                                       NaT   
waiting_for_treatment_event_count                                                        0   
treatment                                                         susceptible_to_treatment   
no_effect_never_treated_event_count                                                      0   
susceptible_to_treatment_event_count                                                     0   
susceptible_to_treatment_event_time                                                    NaT   
positive_test_event_count                                                                0   
positive_test_event_time                                                               NaT   
full_effect_long_event_count                                                             0   
full_effect_long_event_time                                                            NaT   
full_effect_short_event_count                                                            0   
full_effect_short_event_time                                                           NaT   
waning_effect_long_event_count                                                           0   
waning_effect_long_event_time                                                          NaT   
waning_effect_short_event_time                                                         NaT   
waning_effect_short_event_count                                                          0   
no_effect_after_long_event_time                     

In [ ]:
# Simulant 48482 got a negative BBBM test on step 0, then a PET test on
# step 1. They progressed to MCI on step 1, which is why they were
# eligible for both tests.
pd.concat([tpop0.loc[48482], tpop.loc[48482]], axis=1, keys=[0, 1], names=['step'])

step,0,1
tracked,True,True
entrance_time,2029-12-22 00:00:00,2029-12-22 00:00:00
sex,Male,Male
alive,alive,alive
age,75.7913,76.289589
exit_time,NaT,NaT
location,NaN,NaN
cause_of_death,not_dead,not_dead
years_of_life_lost,0.0,0.0
alzheimers_disease_and_other_dementias,alzheimers_blood_based_biomarker_state,alzheimers_mild_cognitive_impairment_state


In [ ]:
# Exactly how much do people age on each time step? Looks like the
# increase is equal to the time step, assuming a year is exactly 365.25
# days. NOTE: If the value of a year doesn't match the mean length of a
# year for the Gregorian calendar (365.2425 days), then eventually
# people's ages will be out of sync with the calendar year
(-73.90981 + 74.408099) * 365.25

182.0000572500051

In [ ]:
# Choose any simulant who is currently alive, and see how much they aged
# on the last time step: Simulant 8 is still alive at the current time.
# Looks like if we assume a year is 365.25 days, then the increase is
# exactly 182 days.
(bpop.loc[8, 'age'] - bpop0.loc[8, 'age']) * 365.25

182.00000000000102

# Check that everyone who got a CSF or PET test on step 1 started in BBBM in step 0 and moved to MCI in step 1

In [134]:
bpop0.loc[got_csf_pet_at_time_1, 'alzheimers_disease_and_other_dementias'].unique()

array(['alzheimers_blood_based_biomarker_state'], dtype=object)

In [135]:
bpop.loc[got_csf_pet_at_time_1, 'alzheimers_disease_and_other_dementias'].unique()

array(['alzheimers_mild_cognitive_impairment_state'], dtype=object)